In [ ]:
# API
# https://asprise.com/ocr/api/docs/html/receipt-ocr.html#multiple-countries

In [ ]:
# pip install fuzzywuzzy
# pip install python-Levenshtein
# pip install requests
# pip install PySimpleGUI

In [1]:
import os
import sys
import json
import nltk
import requests
import pandas as pd
import tkinter as Tk
import PySimpleGUI as sg

from fuzzywuzzy import fuzz, process

from tkinter import *
from tkinter import filedialog
from tkinter.filedialog import askopenfilename

from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
# Testing Api Working Anot

# OCR API endpoint
receipt_ocr_endpoint = 'https://ocr.asprise.com/api/v1/receipt'

# Modify this to use your own file if necessary
image_file = "sample1.jpg"

# Send POST request to OCR API
r = requests.post(receipt_ocr_endpoint, data={
    'client_id': 'TEST',        # Use 'TEST' for testing purpose
    'recognizer': 'auto',       # can be 'US', 'CA', 'JP', 'SG' or 'auto'
    'ref_no': 'ocr_python_123', # optional caller provided ref code
    }, 
    files={"file": open(image_file, "rb")})

# Load JSON response
json_response = json.loads(r.text)

# Extract values from JSON response
receipt = json_response["receipts"][0]
merchant_name = receipt["merchant_name"]
merchant_address = receipt["merchant_address"]
merchant_phone = receipt["merchant_phone"]
mall = receipt["mall"]
receipt_no = receipt["receipt_no"]
items = receipt["items"]
currency = receipt["currency"]
total = receipt["total"]
tax = receipt["tax"]
payment_method = receipt["payment_method"]
date = receipt["date"]

# Print extracted values
print("Merchant Name:", merchant_name)
print("Merchant Address:", merchant_address)
print("Merchant Phone:", merchant_phone)
print("Mall:", mall)
print("Receipt Number:", receipt_no)
print("Items:", items)
print("Currency:", currency)
print("Total:", total)
print("Tax:", tax)
print("Payment Method:", payment_method)
print("Date:", date)

### Defined Functions

In [2]:
# Function to open file dialog and get the file path
def browseFile():
    # Prompt user to select a JPG file
    file_path = filedialog.askopenfilename(filetypes=[("JPG files", "*.jpg")])
    
    if not file_path:
        sg.popup("No File Selected.")
        return None
    else:
        return file_path


def addReceipt(image_file):
    # OCR API endpoint
    receipt_ocr_endpoint = 'https://ocr.asprise.com/api/v1/receipt'

    # Send POST request to OCR API
    r = requests.post(receipt_ocr_endpoint, data={
        'client_id': 'TEST',        # Use 'TEST' for testing purpose
        'recognizer': 'AUTO',       # can be 'US', 'CA', 'JP', 'SG' or 'auto'
        'ref_no': 'ocr_python_123', # optional caller provided ref code
        }, 
        files={"file": open(image_file, "rb")})

    # Load JSON response
    json_response = json.loads(r.text)
    
    # Extract values from JSON response
    receipt = json_response["receipts"][0]
    merchant_name = receipt["merchant_name"]
    merchant_address = receipt["merchant_address"]
    merchant_phone = receipt["merchant_phone"]
    mall = receipt["mall"]
    receipt_no = receipt["receipt_no"]
    items = receipt["items"]
    currency = receipt["currency"]
    total = receipt["total"]
    tax = receipt["tax"]
    payment_method = receipt["payment_method"]
    date = receipt["date"]

    # Store the extracted values to new_receipt dictionary
    new_receipt = {
        "Merchant Name": merchant_name,
        "Merchant Address": merchant_address,
        "Merchant Phone": merchant_phone,
        "Mall": mall,
        "Receipt Number": receipt_no,
        "Items": items,
        "Currency": currency,
        "Total": total,
        "Tax": tax,
        "Payment Method": payment_method,
        "Date": date
    }
    
    headers = ['Merchant Name', 'Merchant Address', 'Merchant Phone', 'Mall', 'Receipt Number', 'Items', 'Currency', 'Total', 'Tax', 'Payment Method', 'Date']
    df = pd.DataFrame([new_receipt], columns=headers)

    layout = [[sg.Table(values=df.values.tolist(), headings=headers, justification='center')], 
              [sg.Column([[sg.Button("Add", size=(10, 1), expand_x=True)]], element_justification='center', expand_x=True),
               sg.Column([[sg.Button("Cancel", size=(10, 1), expand_x=True)]], element_justification='right', expand_x=True)]]
    window = sg.Window('New Receipt: ', layout)
    event, values = window.read()
    window.close()
    
    if event == "Add":  
        data.append(new_receipt)
        
        # Append data to existing JSON file or create a new one
        with open("receipts.json", "a") as f:
            json.dump(new_receipt, f)
            f.write('\n')  # Add newline to separate entries in the JSON file
        
        sg.popup("Successfully Added.")
    else:
        sg.popup("Addition Canceled by User.")

In [3]:
def search_merchant_name(merchant_name, data):
    matching_entities_name = []
    for obj in data:
        if "Merchant Name" in obj and obj["Merchant Name"].lower() == merchant_name.lower():
            matching_entities_name.append(obj)
    return matching_entities_name


def similarity_search(merchant_name, data, threshold=75):
    similar_entities = []
    for obj in data:
        if "Merchant Name" in obj:
            similarity_score = fuzz.partial_ratio(merchant_name.lower(), obj["Merchant Name"].lower())
            if similarity_score >= threshold:
                similar_entities.append(obj)
    return similar_entities


def search_receipt_date(receipt_date, data):
    matching_entities_date = []
    for obj in data:
        if "Date" in obj and obj["Date"] == receipt_date:
            matching_entities_date.append(obj)
    return matching_entities_date

In [13]:
def searchByName():
    layout = [
        [sg.Text("Enter Merchant Name: "), sg.InputText(key="-MERCHANT_NAME-", enable_events=True)],
        [sg.Listbox(values=[merchant["Merchant Name"] for merchant in data], size=(70, 7), key="-MERCHANT_LIST-", enable_events=True, bind_return_key=True, select_mode=sg.LISTBOX_SELECT_MODE_SINGLE)], 
        [sg.Column([[sg.Button("OK", size=(10, 1), expand_x=True)]], element_justification='center', expand_x=True),
         sg.Column([[sg.Button("Cancel", size=(10, 1), expand_x=True)]], element_justification='right', expand_x=True)]
    ]
    window = sg.Window("Enter Merchant Name", layout)
    clicked_ok = False
    while True:
        event, values = window.read()
        if event in (sg.WIN_CLOSED, "Cancel"):
            break
        if event == "-MERCHANT_NAME-":
            search_term = values["-MERCHANT_NAME-"].lower()
            matched_merchants = [merchant for merchant in data if search_term in merchant["Merchant Name"].lower()]
            window["-MERCHANT_LIST-"].update([merchant["Merchant Name"] for merchant in matched_merchants])
        elif event == "-MERCHANT_LIST-":
            if len(values["-MERCHANT_LIST-"]) > 0:
                selected_merchant_name = values["-MERCHANT_LIST-"][0]
                window["-MERCHANT_NAME-"].update(selected_merchant_name)
        if event == "OK":
            clicked_ok = True
            break
    window.close()

    if clicked_ok:
        merchant_name = values["-MERCHANT_NAME-"]
        matching_entities_name = search_merchant_name(merchant_name, data)
        
        if not matching_entities_name:
            sg.popup("No Exact Entities Found.")
            
            # Perform similarity search
            similar_entities = similarity_search(merchant_name, data)
            if similar_entities:
                headers = ['Merchant Name', 'Merchant Address', 'Merchant Phone', 'Mall', 'Receipt Number', 'Items', 'Currency', 'Total', 'Tax', 'Payment Method', 'Date']
                df = pd.DataFrame([obj for obj in similar_entities], columns=headers)

                layout = [[sg.Table(values=df.values.tolist(), headings=headers, justification='center')], 
                          [sg.Column([[sg.Button("OK", size=(10, 1), expand_x=True)]], element_justification='center', expand_x=True)]]
                window = sg.Window('Similar Entities Found: ', layout)
                event, values = window.read()
                window.close()
            else:
                sg.popup("No Similar Entities Found.")
        else:
            headers = ['Merchant Name', 'Merchant Address', 'Merchant Phone', 'Mall', 'Receipt Number', 'Items', 'Currency', 'Total', 'Tax', 'Payment Method', 'Date']
            df = pd.DataFrame([obj for obj in matching_entities_name], columns=headers)

            layout = [[sg.Table(values=df.values.tolist(), headings=headers, justification='center')], 
                      [sg.Column([[sg.Button("OK", size=(10, 1), expand_x=True)]], element_justification='center', expand_x=True)]]
            window = sg.Window('Entities Found: ', layout)
            event, values = window.read()
            window.close()
    else:
        sg.popup("No Merchant Name Entered.")
        window.close()


def searchByDate():
    layout = [
        [sg.InputText(key="-RECEIPT_DATE-"),
         sg.CalendarButton("Choose Receipt Date", close_when_date_chosen=True, target="-RECEIPT_DATE-", format='%Y-%m-%d')],
        [sg.Column([[sg.Button("OK", size=(10, 1), expand_x=True)]], element_justification='center', expand_x=True),
         sg.Column([[sg.Button("Cancel", size=(10, 1), expand_x=True)]], element_justification='right', expand_x=True)]
    ]
    window = sg.Window("Choose Receipt Date", layout)
    event, values = window.read()
    window.close()

    if event == "OK":
        receipt_date = values["-RECEIPT_DATE-"]
        matching_entities_date = search_receipt_date(receipt_date, data)
        if matching_entities_date:
            headers = ['Merchant Name', 'Merchant Address', 'Merchant Phone', 'Mall', 'Receipt Number', 'Items', 'Currency', 'Total', 'Tax', 'Payment Method', 'Date']
            df = pd.DataFrame([obj for obj in matching_entities_date], columns=headers)

            layout = [[sg.Table(values=df.values.tolist(), headings=headers, justification='center')], 
                      [sg.Column([[sg.Button("OK", size=(10, 1), expand_x=True)]], element_justification='center', expand_x=True)]]
            window = sg.Window('Entities Found: ', layout)
            event, values = window.read()
            window.close()
        else:
            sg.popup("No Exact Entites Found.")
    else:
        sg.popup("No Date Chosen.")
        window.close()

In [14]:
def deleteByName():
    layout = [
        [sg.Text("Enter Merchant Name: "), sg.InputText(key="-MERCHANT_NAME-", enable_events=True)],
        [sg.Listbox(values=[merchant["Merchant Name"] for merchant in data], size=(70, 7), key="-MERCHANT_LIST-", enable_events=True, bind_return_key=True, select_mode=sg.LISTBOX_SELECT_MODE_SINGLE)], 
        [sg.Column([[sg.Button("OK", size=(10, 1), expand_x=True)]], element_justification='center', expand_x=True),
         sg.Column([[sg.Button("Cancel", size=(10, 1), expand_x=True)]], element_justification='right', expand_x=True)]
    ]
    window = sg.Window("Enter Merchant Name", layout)
    clicked_ok = False
    while True:
        event, values = window.read()
        if event in (sg.WIN_CLOSED, "Cancel"):
            break
        if event == "-MERCHANT_NAME-":
            search_term = values["-MERCHANT_NAME-"].lower()
            matched_merchants = [merchant for merchant in data if search_term in merchant["Merchant Name"].lower()]
            window["-MERCHANT_LIST-"].update([merchant["Merchant Name"] for merchant in matched_merchants])
        elif event == "-MERCHANT_LIST-":
            if len(values["-MERCHANT_LIST-"]) > 0:
                selected_merchant_name = values["-MERCHANT_LIST-"][0]
                window["-MERCHANT_NAME-"].update(selected_merchant_name)
        if event == "OK":
            clicked_ok = True
            break
    window.close()

    if clicked_ok:
        merchant_name = values["-MERCHANT_NAME-"]

        # Create a empty list called 'delete_entities_name'
        delete_entities_name = []
        for obj in data:
            if "Merchant Name" in obj and obj["Merchant Name"].lower() == merchant_name.lower():
                delete_entities_name.append(obj)
        
        if delete_entities_name:
            headers = ['Merchant Name', 'Merchant Address', 'Merchant Phone', 'Mall', 'Receipt Number', 'Items', 'Currency', 'Total', 'Tax', 'Payment Method', 'Date']
            df = pd.DataFrame([obj for obj in delete_entities_name], columns=headers)

            layout = [[sg.Table(values=df.values.tolist(), headings=headers, justification='center')], 
                      [sg.Column([[sg.Button("Delete", size=(10, 1), expand_x=True)]], element_justification='center', expand_x=True),
                       sg.Column([[sg.Button("Cancel", size=(10, 1), expand_x=True)]], element_justification='right', expand_x=True)]]
            window = sg.Window('Entities Found: ', layout)
            event, values = window.read()
            window.close()
    
            if event == "Delete":  
                for obj in delete_entities_name:
                    data.remove(obj)
                
                # Update the receipts.json file
                with open('receipts.json', 'w') as f:
                    for obj in data:
                        f.write(json.dumps(obj) + '\n')
                        
                sg.popup("Successfully Deleted.")
            else:
                sg.popup("Deletion Canceled by User.")
        else:
            sg.popup("No Exact Entities Found.")
    else:
        sg.popup("No Merchant Name Entered.")
        window.close()
        
        
def deleteByDate():
    layout = [
        [sg.InputText(key="-RECEIPT_DATE-"),
         sg.CalendarButton("Choose Receipt Date", close_when_date_chosen=True, target="-RECEIPT_DATE-", format='%Y-%m-%d')],
        [sg.Column([[sg.Button("OK", size=(10, 1), expand_x=True)]], element_justification='center', expand_x=True),
         sg.Column([[sg.Button("Cancel", size=(10, 1), expand_x=True)]], element_justification='right', expand_x=True)]
    ]
    window = sg.Window("Choose Receipt Date", layout)
    event, values = window.read()
    window.close()
    
    if event == "OK":
        receipt_date = values["-RECEIPT_DATE-"]

        delete_entities_date = []
        for obj in data:
            if "Date" in obj and obj["Date"] == receipt_date:
                delete_entities_date.append(obj)
        
        if delete_entities_date:
            headers = ['Merchant Name', 'Merchant Address', 'Merchant Phone', 'Mall', 'Receipt Number', 'Items', 'Currency', 'Total', 'Tax', 'Payment Method', 'Date']
            df = pd.DataFrame([obj for obj in delete_entities_date], columns=headers)

            layout = [[sg.Table(values=df.values.tolist(), headings=headers, justification='center')], 
                      [sg.Column([[sg.Button("Delete", size=(10, 1), expand_x=True)]], element_justification='center', expand_x=True),
                       sg.Column([[sg.Button("Cancel", size=(10, 1), expand_x=True)]], element_justification='right', expand_x=True)]]
            window = sg.Window('Entities Found: ', layout)
            event, values = window.read()
            window.close()
    
            if event == "Delete":  
                for obj in delete_entities_date:
                    data.remove(obj)
                
                # Update the receipts.json file
                with open('receipts.json', 'w') as f:
                    for obj in data:
                        f.write(json.dumps(obj) + '\n')
                        
                sg.popup("Successfully Deleted.")
            else:
                sg.popup("Deletion Canceled by User.")
        else:
            sg.popup("No Exact Entities Found.")
    else:
        sg.popup("No Date Chosen.")
        window.close()

In [15]:
def viewAllRecords():
    if data:
        headers = ['Merchant Name', 'Merchant Address', 'Merchant Phone', 'Mall', 'Receipt Number', 'Items', 'Currency', 'Total', 'Tax', 'Payment Method', 'Date']
        
        # Validate keys of each dictionary is consistent
        for obj in data:
            if set(obj.keys()) != set(headers):
                sg.Popup("Error: Inconsistent Keys In Dictionaries.")
                return
        
        df = pd.DataFrame([obj for obj in data], columns=headers)

        layout = [
            [sg.Table(values=df.values.tolist(), headings=headers, justification='center', key='-TABLE-')],
            [sg.Button('Sort by Merchant Name', key='-SORTNAME-', size=(20, 1), expand_x=True), 
             sg.Button('Sort by Receipt Date', key='-SORTDATE-', size=(20, 1), expand_x=True), 
             sg.Button('Export to CSV', key='-EXPORT-', size=(20, 1), expand_x=True),
             sg.Button('OK', size=(20, 1), key='-OK-', expand_x=True)]
        ]

        
        window = sg.Window('All Records in the Database: ', layout)
        while True:
            event, values = window.read()
            if event == sg.WINDOW_CLOSED or event == '-OK-':
                break
            elif event == '-SORTNAME-':
                sorted_data = sorted(data, key=lambda x: x['Merchant Name'])
                df = pd.DataFrame([obj for obj in sorted_data], columns=headers)
                window['-TABLE-'].update(values=df.values.tolist())
            elif event == '-SORTDATE-':
                sorted_data = sorted(data, key=lambda x: pd.Timestamp(x['Date']) if x['Date'] else pd.Timestamp.min, reverse=True)
                df = pd.DataFrame([obj for obj in sorted_data], columns=headers)
                window['-TABLE-'].update(values=df.values.tolist())
            elif event == '-EXPORT-':
                filename = sg.popup_get_file('Save As', save_as=True, default_extension='.csv')
                if filename:
                    df.to_csv(filename, index=False)
                    sg.popup("Successfully Saved.")
        window.close()
    else:
        sg.popup("No Records Found in the Database.")

### Main Function

In [16]:
# Create 'receipts.json' file if the file doesn't exist
filename = 'receipts.json'
if not os.path.isfile(filename):
    open(filename, 'w').close()

# Load data from 'receipts.json' file to a list of dictionaries called 'data'
with open('receipts.json', 'r') as f:
    data = [json.loads(line) for line in f]

# Define PySimpleGUI layout
layout = [
    [sg.Column([[sg.Text("Receipt Document Managment System", font=("Arial", 14, "bold italic"), text_color='#ADD8E6')]], expand_x=True, element_justification="center")],
    [sg.Radio("Add Receipt Document", "menu", default=True, key="-ADD-")],
    [sg.Radio("Search by Merchant Name", "menu", default=True, key="-SEARCH_BY_NAME-")],
    [sg.Radio("Search by Receipt Date", "menu", default=True, key="-SEARCH_BY_DATE-")],
    [sg.Radio("Delete Entities by Merchant Name", "menu", default=True, key="-DELETE_BY_NAME-")],
    [sg.Radio("Delete Entities by Receipt Date", "menu", default=True, key="-DELETE_BY_DATE-")],
    [sg.Radio("View All Records in the Database", "menu", default=True, key="-VIEW-")],
    [sg.Column([[sg.Button("Exit", size=(10, 1), expand_x=True)]], element_justification='center', expand_x=True),
     sg.Column([[sg.Button("Choose", size=(10, 1), expand_x=True)]], element_justification='right', expand_x=True)]
]

# Define PySimpleGUI window
window = sg.Window("Receipt Manager", layout)

# Start event loop for PySimpleGUI window
while True:
    event, values = window.read()

    # Add receipt document
    if event == "Choose" and values["-ADD-"]:
        # Create a Tkinter window
        root = Tk()
        root.withdraw()

        file_path = browseFile()
        if file_path is not None:
            addReceipt(file_path)
            
    # Search by merchant name
    elif event == "Choose" and values["-SEARCH_BY_NAME-"]:
        searchByName()
            
    # Search by receipt date
    elif event == "Choose" and values["-SEARCH_BY_DATE-"]:
        searchByDate()
    
    # Delete entities by merchant name
    elif event == "Choose" and values["-DELETE_BY_NAME-"]:
        deleteByName()
        
    # Delete entities by receipt date
    elif event == "Choose" and values["-DELETE_BY_DATE-"]:
        deleteByDate()

    # View all records in the database
    elif event == "Choose" and values["-VIEW-"]:
        viewAllRecords()

    # Exit using exit button
    elif event == "Exit":
        # Show confirmation popup
        confirm_exit = sg.popup_yes_no("Are You Sure You Want To Exit?")
        if confirm_exit == "Yes":
            break
            
    # Exit using close button
    elif event == sg.WIN_CLOSED or event == "Exit":
        # Show confirmation popup
        confirm_exit = sg.popup_yes_no("Are You Sure You Want To Exit?")
        if confirm_exit == "Yes":
            break
        elif confirm_exit == "No" or event == sg.WIN_CLOSED:
            # Define PySimpleGUI layout
            layout = [
                [sg.Column([[sg.Text("Receipt Document Managment System", font=("Arial", 14, "bold italic"), text_color='#ADD8E6')]], expand_x=True, element_justification="center")],
                [sg.Radio("Add Receipt Document", "menu", default=True, key="-ADD-")],
                [sg.Radio("Search by Merchant Name", "menu", default=True, key="-SEARCH_BY_NAME-")],
                [sg.Radio("Search by Receipt Date", "menu", default=True, key="-SEARCH_BY_DATE-")],
                [sg.Radio("Delete Entities by Merchant Name", "menu", default=True, key="-DELETE_BY_NAME-")],
                [sg.Radio("Delete Entities by Receipt Date", "menu", default=True, key="-DELETE_BY_DATE-")],
                [sg.Radio("View All Records in the Database", "menu", default=True, key="-VIEW-")],
                [sg.Column([[sg.Button("Exit", size=(10, 1), expand_x=True)]], element_justification='center', expand_x=True),
                 sg.Column([[sg.Button("Choose", size=(10, 1), expand_x=True)]], element_justification='right', expand_x=True)]
            ]
            window = sg.Window("Receipt Manager", layout)

# Close PySimpleGUI window
window.close()